In [21]:
import os.path
import pandas as pd
import orodja
%matplotlib inline

vsi_kovanci = orodja.seznam_kovancev()

pd.options.display.max_rows = 20

def vrni_kovanec(ime):
    return pd.read_csv('./podatki/'+ ime + '.csv')

posamezni_podatki = []
for kovanec in vsi_kovanci:
    if not os.path.isfile('./podatki/'+ kovanec + '.csv'):
        continue
    kovanec_df = vrni_kovanec(kovanec)
    # Dodajanje kolone z imenom kovanca
    kovanec_df['kovanec'] = pd.Series([kovanec] * len(kovanec_df.index), index=kovanec_df.index)
    posamezni_podatki.append(kovanec_df)
kovanci = pd.concat(posamezni_podatki)

In [24]:
kovanci

,datum,zacetna,najnizja,najvisja,koncna,volumen,kapital,kovanec
0,"Jan 01, 2017",963.660000,1003.080000,958.700000,998.330000,147775000.0,1.549120e+10,bitcoin
1,"Jan 02, 2017",998.620000,1031.390000,996.700000,1021.750000,222185000.0,1.605510e+10,bitcoin
2,"Jan 03, 2017",1021.600000,1044.080000,1021.600000,1043.840000,185168000.0,1.642660e+10,bitcoin
3,"Jan 04, 2017",1044.400000,1159.420000,1044.400000,1154.730000,344946000.0,1.679540e+10,bitcoin
4,"Jan 05, 2017",1156.730000,1191.100000,910.420000,1013.380000,510199000.0,1.860400e+10,bitcoin
5,"Jan 06, 2017",1014.240000,1046.810000,883.940000,902.200000,351876000.0,1.631410e+10,bitcoin
6,"Jan 07, 2017",903.490000,908.590000,823.560000,908.590000,279550000.0,1.453440e+10,bitcoin
7,"Jan 08, 2017",908.170000,942.720000,887.250000,911.200000,158715000.0,1.461140e+10,bitcoin
8,"Jan 09, 2017",913.240000,913.690000,879.810000,902.830000,141877000.0,1.469490e+10,bitcoin
9,"Jan 10, 2017",902.440000,914.870000,901.060000,907.680000,115808000.0,1.452260e+10,bitcoin
